In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../src")
import xgboost as xgb
import pandas as pd
import numpy as np

from features.UserJoin import (
    UserJoin,
    submit,
    diff,
    plt_month,
    plt_day,
    load_ids,
    info,
    infot,
    f1score,
    gen_day_hol_n,
)

import config as C
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns
from xgboost import plot_tree, plot_importance
import random
from pathlib import Path

import warnings

warnings.filterwarnings("ignore")

from tqdm import tqdm
import itertools

sns.set_style("ticks", {"axes.grid": True})

pd.options.display.max_columns = 500
# pd.options.display.max_rows = 20
# pd.options.display.min_rows = 20
plt.style.use("ggplot")

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
def score(right, wrong):
    right = int(right)
    wrong = int(wrong)
    true_num = 37
    y_true = np.zeros(15379)
    y_pred = np.zeros(15379)
    y_true[:true_num] = 1
    y_pred[:right] = 1
    if wrong:
        y_pred[-wrong:] = 1
    score = f1_score(y_true, y_pred, average="macro")
    return score

In [11]:
score(36, 8)

0.9442977547420617

In [2]:
ds = UserJoin()

In [3]:
ids = C.true_ids
ds.train.query("id.isin(@ids)")

,ELEC_TYPE_NAME,VOLT_NAME,RUN_CAP,label,kwh,kwh_cal,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4,pr2,pr3,pr4,2_3,2_4,3_4,daycv,kwh_holiday_mean_1,kwh_cal_holiday_mean_1,kwh_pap_r2_holiday_mean_1,kwh_pap_r3_holiday_mean_1,kwh_pap_r4_holiday_mean_1,pr2_holiday_mean_1,pr3_holiday_mean_1,pr4_holiday_mean_1,2_3_holiday_mean_1,2_4_holiday_mean_1,3_4_holiday_mean_1,daycv_holiday_mean_1,kwh_holiday_mean_0,kwh_cal_holiday_mean_0,kwh_pap_r2_holiday_mean_0,kwh_pap_r3_holiday_mean_0,kwh_pap_r4_holiday_mean_0,pr2_holiday_mean_0,pr3_holiday_mean_0,pr4_holiday_mean_0,2_3_holiday_mean_0,2_4_holiday_mean_0,3_4_holiday_mean_0,daycv_holiday_mean_0,kwh_holiday_std_1,kwh_cal_holiday_std_1,kwh_pap_r2_holiday_std_1,kwh_pap_r3_holiday_std_1,kwh_pap_r4_holiday_std_1,pr2_holiday_std_1,pr3_holiday_std_1,pr4_holiday_std_1,2_3_holiday_std_1,2_4_holiday_std_1,3_4_holiday_std_1,daycv_holiday_std_1,kwh_holiday_std_0,kwh_cal_holiday_std_0,kwh_pap_r2_holiday_std_0,kwh_pap_r3_holiday_std_0,kwh_pap_r4_holiday_std_0,pr2_holiday_std_0,pr3_holiday_std_0,pr4_holiday_std_0,2_3_holiday_std_0,2_4_holiday_std_0,3_4_holiday_std_0,daycv_holiday_std_0,kwh_holiday_diff,kwh_cal_holiday_diff,kwh_pap_r2_holiday_diff,kwh_pap_r3_holiday_diff,kwh_pap_r4_holiday_diff,pr2_holiday_diff,pr3_holiday_diff,pr4_holiday_diff,2_3_holiday_diff,2_4_holiday_diff,3_4_holiday_diff,daycv_holiday_diff,kwh_holiday_cv,kwh_cal_holiday_cv,kwh_pap_r2_holiday_cv,kwh_pap_r3_holiday_cv,kwh_pap_r4_holiday_cv,kwh_workday_cv,kwh_cal_workday_cv,kwh_pap_r2_workday_cv,kwh_pap_r3_workday_cv,kwh_pap_r4_workday_cv,kwh_cv,kwh_cal_cv,kwh_pap_r2_cv,kwh_pap_r3_cv,kwh_pap_r4_cv,pq_f,pq_g,pq_p,pq_z,pp,pf,pg,p_f,p_g,f_g,monthcv,isminer,cv2_all,cv3_all,cv4_all,cv_all,cv2_holiday,cv3_holiday,cv4_holiday,cv_holiday,cv2_workday,cv3_workday,cv4_workday,cv_workday
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
362400993,乡村居民生活用电,交流10kV,24.0,test,291.919065,291.919159,118.950561,68.370374,104.598224,0.469695,0.152610,0.377695,0.317688,0.093580,0.225527,0.491478,284.346842,284.347544,116.428070,65.941579,101.977895,0.470752,0.152003,0.377245,0.319299,0.094758,0.225489,0.493088,300.5514,300.5508,121.8262,71.1392,107.5854,0.468489,0.153302,0.378209,0.315851,0.092237,0.225570,0.489642,172.528468,172.528598,46.056290,79.553927,53.349847,0.127601,0.167453,0.040412,0.294371,0.086891,0.207248,0.453050,179.492950,179.493175,46.820998,83.129241,55.304185,0.126990,0.167787,0.042053,0.293875,0.085282,0.208352,0.452246,-16.204558,-16.203256,-5.398130,-5.197621,-5.607505,0.002262,-0.001298,-0.000964,0.003448,0.002521,-0.000082,0.003446,0.606754,0.606753,0.395577,1.206430,0.523151,0.597212,0.597214,0.384326,1.168543,0.514049,0.600064,0.600064,0.389023,1.183258,0.517094,3737.476190,3272.190476,2088.904762,9098.571429,0.142729,0.477138,0.380133,0.335077,0.237590,0.099127,0.518235,miner,0.022187,0.004700,0.016295,0.018851,0.010534,0.002034,0.004410,0.007684,0.028102,0.005808,0.021711,0.024506
179418058,普通工业,交流380V,100.0,test,269.280374,269.277570,93.256075,93.764486,82.257009,0.342857,0.345395,0.302402,0.011130,0.051509,0.050600,0.091310,272.554386,272.571930,95.050877,95.042105,82.478947,0.349645,0.349373,0.300982,0.010311,0.048662,0.048391,0.086940,265.5480,265.5220,91.2100,92.3080,82.0040,0.335120,0.340861,0.304020,0.012065,0.054755,0.053118,0.096291,111.071753,111.058789,38.207806,38.077754,35.346033,0.012917,0.011858,0.020054,0.010230,0.031582,0.030310,0.051872,120.578995,120.583346,42.263688,41.514949,38.134996,0.061461,0.052120,0.069028,0.022741,0.078793,0.058046,0.121703,7.006386,7.049930,3.840877,2.734105,0.474947,0.014525,0.008512,-0.003037,-0.001754,-0.006092,-0.004727,-0.009352,0.407521,0.407448,0.401972,0.400641,0.428546,0.454076,0.454137,0.463367,0.449744,0.465038,0.427483,0.427473,0.429013,0.421852,0.443747,2898.285714,2523.238095,2915.380952,8336.904762,0.352463,0.349637,0.297900,0.004221,0.054563,0.051737,0.092469,miner,0.132102,0.124734,0.098951,0.111241,0.077772,0.075

In [8]:
ds.train.groupby(["ELEC_TYPE_NAME", "VOLT_NAME"]).count()

RUN_CAP  label    kwh  kwh_cal  kwh_pap_r2  \
ELEC_TYPE_NAME VOLT_NAME                                               
乡村居民生活用电       交流10kV          34     34     34       34          34   
               交流220V        7306   7306   7306     7306        7306   
               交流380V        1762   1762   1762     1762        1762   
               交流6kV            1      1      1        1           1   
农业生产用电         交流10kV          10     10     10       10          10   
商业用电           交流10kV          32     32     32       32          32   
               交流220V           7      7      7        7           7   
               交流380V          20     20     20       20          20   
城镇居民生活用电       交流10kV          29     29     29       29          29   
               交流220V       10663  10663  10663    10663       10663   
               交流380V        3490   3490   3490     3490        3490   
大工业用电          交流10kV          21     21     21       21          21   
学校教学和学生生活用电    交流10kV           5      5      5        5           5   
               交流220V           7      7      7        7           7   
               交流380V          14     14     14       14          14   
居民生活用电         交流10kV           3      3      3        3           3   
               交流220V          47     47     47       47          47   
               交流380V          26     26     26       26          26   
普通工业           交流10kV          77     77     77       77          77   
               交流380V          10     10     10       10          10   
非居民照明          交流10kV          35     35     35       35          35   
               交流220V         498    498    498      498         498   
               交流380V         116    116    116      116         116   
非工业            交流10kV           3      3      3        3           3   
               交流380V           1      1      1        1           1   

                          kwh_pap_r3  kwh_pap_r4    pr2    pr3    pr4    2_3  \
ELEC_TYPE_NAME VOLT_NAME                                                       
乡村居民生活用电       交流10kV             34          34     34     34     34     34   
               交流220V           7306        7306   7306   7306   7306   7306   
               交流380V           1762        1762   1762   1762   1762   1762   
               交流6kV               1           1      1      1      1      1   
农业生产用电         交流10kV             10          10     10     10     10     10   
商业用电           交流10kV             32          32     32     32     32     32   
               交流220V              7           7      7      7      7      7   
               交流380V             20          20     20     20     20     20   
城镇居民生活用电       交流10kV             29          29     29     29     29     29   
               交流220V          10663       10663  10663  10663  10663  10663   
               交流380V           3490        3490   3490   3490   3490   3490   
大工业用电          交流10kV             21          21     21     21     21     21   
学校教学和学生生活用电    交流10kV              5           5      5      5      5      5   
               交流220V              7           7      7      7      7      7   
               交流380V             14          14     14     14     14     14   
居民生活用电         交流10kV              3           3      3      3      3      3   
               交流220V             47          47     47     47     47     47   
               交流380V             26          26     26     26     26     26   
普通工业           交流10kV             77          77     77     77     77     77   
               交流380V             10          10     10     10     10     10   
非居民照明          交流10kV             35          35     35     35     35     35   
               交流220V            498         498    498    498    498    498   
               交流380V            116         116    116    116    116    116   
非工业            交流10kV              3           3      3      3      3      3   
 